# Pandas et itérateurs - correction

[pandas](http://pandas.pydata.org/) a tendance a prendre beaucoup d'espace mémoire pour charger les données, environ trois fois plus que sa taille sur disque.  Quand la mémoire n'est pas assez grande, que peut-on faire ?

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()
import pandas
df = pandas.DataFrame(data.data)
df.column = "X1 X2 X3 X4".split()
df["target"] = data.target
df.to_csv("iris.txt", sep="\t", index=False)

## Exercice 1 : itérer sur un grand fichier

Le paramètre *iterator* de la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) sert à parcourir un fichier par blocs dont la taille est définie par le paramètres *chunksize*. La fonction [read_csv](http://www.xavierdupre.fr/app/pandas_streaming/helpsphinx/pandas_streaming/df/dataframe.html#pandas_streaming.df.dataframe.StreamingDataFrame.read_csv) implémente ce mécanisme.

In [ ]:
for df in pandas.read_csv("iris.txt", sep="\t", iterator=True, chunksize=60):
    print(df.shape)

(60, 5)
(60, 5)
(30, 5)


## Exercice 2 : split train test

La solution proposée est implémentée par [train_test_split](http://www.xavierdupre.fr/app/pandas_streaming/helpsphinx/pandas_streaming/df/dataframe.html#pandas_streaming.df.dataframe.StreamingDataFrame.train_test_split).

In [ ]:
from pandas_streaming.df import StreamingDataFrame
sdf = StreamingDataFrame.read_csv("iris.txt", sep="\t", iterator=True, chunksize=60)
sdf.train_test_split("iris_{}.txt")

['iris_train.txt', 'iris_test.txt']

## Exercice 3 : stratify ?

Le paramètre *stratify* est intéressant pour un problème de classification et quand une classes et sous-représentée. Il est fort probable  que cette classe ne soit pas assez représentée dans l'un des deux jeux et c'est pourquoi il existe une option pour imposer un nombre d'exemples de cette dans chaque des deux jeux (train, test). La qualité des modèles est accrue tout comme la qualité des sondages sur un [échantillonnage stratifié](https://en.wikipedia.org/wiki/Stratified_sampling).

Si jamais tout ces exemples sont placés au début du gros fichier à lire, le programme commence à avoir une fausse idée de la répartition des classes. La seule façon de faire est de faire d'abord une division train/test par classe (indiqué par la variable de stratification) puis de recomposer les bases d'apprentissage et de tests en imposant les proportions voulues.

## Exercice 4 : quelques idées pour un group by ?

La fonction [groupby](http://www.xavierdupre.fr/app/pandas_streaming/helpsphinx/pandas_streaming/df/dataframe.html#pandas_streaming.df.dataframe.StreamingDataFrame.groupby) implémente une façon de faire. Il faut distinguer deux cas possibles. Premier cas, l'agrégation aboutit à un résultat qui tient en mémoire auquel on peut s'en sortir aisément. Dans le second cas, l'agrégation ne tient pas en mémoire et il faudra probablement passer par un fichier intermédiaire...